\begin{aligned}
&u_{t}+u u_{x}-(0.01 / \pi) u_{x x}=0, \quad x \in[-1,1], \quad t \in[0,1] \\
&u(0, x)=-\sin (\pi x) \\
&u(t,-1)=u(t, 1)=0
\end{aligned}

In [ ]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from models_all import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

# Data For PINNs

In [ ]:
N_train = 10000
N_bound = 100

la = np.array([1,1])
lb = np.array([-1,0])
np.random.seed(123)
traindata = lb+(la-lb)*lhs(2,N_train)
np.random.seed(1234)
bound_x = -1+(1-(-1))*lhs(1,N_bound)
np.random.seed(12345)
bound_t = 0+(1-(0))*lhs(1,N_bound)

In [ ]:
#x,t  #u,v
X_train = torch.from_numpy(traindata).float()
x_bound = torch.from_numpy(bound_x).float()
t_bound = torch.from_numpy(bound_t).float()
x = X_train[:,0:1]
t = X_train[:,1:2]

x.requires_grad_()
t.requires_grad_()

## Get Data

In [ ]:
import scipy.io
data = scipy.io.loadmat('./burgers_shock.mat')
t_ = data['t'].flatten()[:,None]
x_ = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T

X, T = np.meshgrid(x_,t_)

X_star = np.hstack((T.flatten()[:,None], X.flatten()[:,None]))
u_star = Exact.flatten()[:,None]  

### Labeled Data

In [ ]:
np.random.seed(1234)
index_t = (lhs(1,10))*len(t_)
index_t = index_t.astype(int).reshape(10,)
t_star = t_[index_t]
index_x = (lhs(1,10))*len(x_)
index_x = index_x.astype(int).reshape(10,)
x_star = x_[index_x]
t_x_star = np.hstack((t_star, x_star))

In [ ]:
u_star_ = Exact[index_t,index_x]
n=10

In [ ]:
gan_data_x_t = t_x_star
gan_data_u = u_star_.reshape(-1,1)
gan_data_x_t = torch.from_numpy(gan_data_x_t).float()
gan_data_u = torch.from_numpy(gan_data_u).float()

### The Image With Labeled Sample

In [ ]:
fig, ax = plt.subplots()
gs0 = gridspec.GridSpec(1, 2)
gs0.update(top=1-0.06, bottom=1-1/3, left=0.15, right=0.85, wspace=0)
ax = plt.subplot(gs0[:, :])
ax.plot(gan_data_x_t[:,0:1], gan_data_x_t[:,1:2],  'kx',alpha=1,markersize = 4)
h = ax.imshow(Exact.T, interpolation='nearest', cmap='rainbow',  extent=[0, 1, -1, 1],
            origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)
plt.show()

# Test Data

In [ ]:
np.random.seed(5678)
index_t = (lhs(1,10000))*len(t_)
index_t = index_t.astype(int).reshape(10000,)
t_star = t_[index_t]
index_x = (lhs(1,10000))*len(x_)
index_x = index_x.astype(int).reshape(10000,)
x_star = x_[index_x]
t_x_star = np.hstack((t_star, x_star))

In [ ]:
u_star_ = Exact[index_t,index_x]

In [ ]:
test_data = t_x_star
test_u = u_star_.reshape(-1,1)
test_data = torch.from_numpy(test_data).float()

In [ ]:
test_u

# Method1
$\text { PINN }$

<div class="alert alert-info">objective function：
$\mathrm{L}_{\text {PINN }}:=\mathrm{L}_f+\lambda_1 \mathrm{~L}_b \text { with } \mathrm{L}_b:=\sum_{i=1}^I \mathrm{~L}_{b_i}$</div>

In [ ]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

PINNs1 = NN_H2(2, 20, 7, 1)
PINNs1.apply(weights_init)
optimizer1 = optim.Adam(PINNs1.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
nIter1 = 10000

loss_all_1 = []
test_loss_1 = []

loss1_value = 1
it = 0
while  loss1_value>0.0001:
    ##### loss_Bi  ######
    E_bound = PINNs1(torch.cat((torch.zeros_like(x_bound),x_bound),1))
    real_bound = -torch.sin(torch.pi*x_bound)
    loss_bound = torch.mean(torch.square(E_bound-real_bound))+\
                 torch.mean(torch.square(PINNs1(torch.cat((t_bound,torch.ones_like(t_bound)),1))))+\
                 torch.mean(torch.square(PINNs1(torch.cat((t_bound,-torch.ones_like(t_bound)),1))))
    
    ##### loss f  ######
    
    E_inside = PINNs1(torch.cat((t,x),1))
    E_x = autograd.grad(outputs=E_inside, inputs=x,
                              grad_outputs=torch.ones_like(E_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    E_xx = autograd.grad(outputs=E_x, inputs=x,
                              grad_outputs=torch.ones_like(E_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    E_t = autograd.grad(outputs=E_inside, inputs=t,
                              grad_outputs=torch.ones_like(E_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]     

    loss_f = torch.mean(torch.square(E_t+E_inside*E_x-0.01/torch.pi*(E_xx)))
     
    #####loss PI#######
    loss = loss_bound+loss_f
    
    loss_all_1.append(loss.item())
    loss1_value = loss.item()
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    #########  test_loss NRMSE  #########
    test_loss = relative_l2(PINNs1(test_data).detach().numpy(),test_u)
    test_loss_1.append(test_loss)
    
    if it % 100 == 0:
        print('It:', it, 'Loss:', loss.item())
    it = it + 1        
loss1_value    

# Results

## Save data

In [ ]:
# np.save('../experimental_data/method_1/test_loss_1',test_loss_1)
# np.save('../experimental_data/method_1/loss_all_1',loss_all_1)
# torch.save(PINNs1,'../saved_model/PINNs1')

## Epoch and NRMSE

In [ ]:
Epochs = len(test_loss_1)
NRMSE = relative_l2(PINNs1(test_data).detach().numpy(),test_u)

print('Epochs:',Epochs,'NRMSE:',NRMSE)